In [ ]:
# !pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
df=pd.read_csv("/content/drive/MyDrive/Datasets/f_sql.csv")
df1=pd.read_csv("/content/drive/MyDrive/Datasets/Updated_Query_Dataset.csv")
df1 = df1.rename(columns={'Question': 'question','SQL_Query': 'sql'})
df=df.drop("Unnamed: 0",axis=1)

In [ ]:
df1

,question,sql
0,What is the average price of items in the data...,SELECT AVG(Price) as avg_price FROM table_name;
1,Which brand has the highest average rating for...,"SELECT Brand, AVG(Rating) as avg_rating FROM t..."
2,What is the most common delivery time for item...,"SELECT deleivery_time, COUNT(deleivery_time) a..."
3,Which seller has the highest average shipment ...,"SELECT seller_name, AVG(shipment_rating) as av..."
4,How many items in the dataset have a discount ...,SELECT COUNT(*) as count_discount_above_50 FRO...
...,...,...
95,How many sellers in the CSV file have a chat r...,SELECT COUNT(*) AS num_sellers FROM sellers WH...
96,What is the distribution of delivery times for...,"SELECT delivery_time, COUNT(*) AS num_products..."
97,Which seller has the highest ratio of seller r...,"SELECT seller_name, seller_rating / chat_respo..."
98,How many products in the CSV file have a disco...,SELECT COUNT(*) AS num_products FROM products ...


In [ ]:
# df=df.drop("context",axis=1)
df

,question,sql
0,How many heads of the departments are older th...,SELECT COUNT(*) FROM head WHERE age > 56
1,"List the name, born state and age of the heads...","SELECT name, born_state, age FROM head ORDER B..."
2,"List the creation year, name and budget of eac...","SELECT creation, name, budget_in_billions FROM..."
3,What are the maximum and minimum budget of the...,"SELECT MAX(budget_in_billions), MIN(budget_in_..."
4,What is the average number of employees of the...,SELECT AVG(num_employees) FROM department WHER...
...,...,...
134927,What time was the match played with a score of...,SELECT Time FROM table WHERE Score = 3-2
134928,On which ground did the team play Aston Villa?,SELECT Ground FROM table WHERE Opponent = asto...
134929,What kind of competition was it at San Siro at...,SELECT Competition FROM table WHERE Ground = s...
134930,What is the total number of decile for the red...,SELECT COUNT Decile FROM table WHERE Name = re...


In [ ]:
# df = df.rename(columns={'answer': 'sql'})
# df

,question,sql
0,How many heads of the departments are older th...,SELECT COUNT(*) FROM head WHERE age > 56
1,"List the name, born state and age of the heads...","SELECT name, born_state, age FROM head ORDER B..."
2,"List the creation year, name and budget of eac...","SELECT creation, name, budget_in_billions FROM..."
3,What are the maximum and minimum budget of the...,"SELECT MAX(budget_in_billions), MIN(budget_in_..."
4,What is the average number of employees of the...,SELECT AVG(num_employees) FROM department WHER...
...,...,...
78572,What time was the match played with a score of...,"SELECT time FROM table_name_35 WHERE score = ""..."
78573,On which ground did the team play Aston Villa?,SELECT ground FROM table_name_83 WHERE opponen...
78574,What kind of competition was it at San Siro at...,SELECT competition FROM table_name_60 WHERE gr...
78575,What is the total number of decile for the red...,SELECT COUNT(decile) FROM table_name_34 WHERE ...


In [ ]:
df_concat = pd.concat([df, df1], axis=0)

# reset index
df_concat = df_concat.reset_index(drop=True)
df_concat

,question,sql
0,How many heads of the departments are older th...,SELECT COUNT(*) FROM head WHERE age > 56
1,"List the name, born state and age of the heads...","SELECT name, born_state, age FROM head ORDER B..."
2,"List the creation year, name and budget of eac...","SELECT creation, name, budget_in_billions FROM..."
3,What are the maximum and minimum budget of the...,"SELECT MAX(budget_in_billions), MIN(budget_in_..."
4,What is the average number of employees of the...,SELECT AVG(num_employees) FROM department WHER...
...,...,...
135027,How many sellers in the CSV file have a chat r...,SELECT COUNT(*) AS num_sellers FROM sellers WH...
135028,What is the distribution of delivery times for...,"SELECT delivery_time, COUNT(*) AS num_products..."
135029,Which seller has the highest ratio of seller r...,"SELECT seller_name, seller_rating / chat_respo..."
135030,How many products in the CSV file have a disco...,SELECT COUNT(*) AS num_products FROM products ...


In [ ]:
df_concat.to_csv("/content/drive/MyDrive/Datasets/final_sql.csv")

In [ ]:
import csv
import torch
from torch.nn.utils.rnn import pad_sequence
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments

# Load CSV file
with open('/content/drive/MyDrive/final_sql.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    qa_pairs = [(row[1], row[2]) for row in reader]
# qa_pairs=(qa_pairs[100000:])

# Define T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Define training and validation datasets
train_dataset = []
val_dataset = []
for i, qa in enumerate(qa_pairs):
    input_text = "question: " + qa[0]
    target_text = qa[1] + " </s>"
    # print(input_text)

    if i % 5 == 0:
        val_dataset.append((input_text, target_text))
    else:
        train_dataset.append((input_text, target_text))

# Define data collator
def data_collator(batch):
    input_ids = [tokenizer.encode(item[0], max_length=512, truncation=True) for item in batch]
    target_ids = [tokenizer.encode(item[1], max_length=512, truncation=True) for item in batch]

    # Pad input and target sequences to the same length
    input_ids = pad_sequence([torch.tensor(x) for x in input_ids], batch_first=True, padding_value=tokenizer.pad_token_id)
    target_ids = pad_sequence([torch.tensor(x) for x in target_ids], batch_first=True, padding_value=tokenizer.pad_token_id)

    return {"input_ids": input_ids, "attention_mask": input_ids.ne(tokenizer.pad_token_id).float(),
            "labels": target_ids[:, :-1], "decoder_attention_mask": target_ids[:, :-1].ne(tokenizer.pad_token_id).float()}

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    # evaluate_during_training=True,
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=5
)

# Define Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

# Start training
trainer.train()

# Save fine-tuned model


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprec

Step,Training Loss
10,9.450300
20,10.020700
30,9.337800
40,8.602600
50,8.268300
60,7.518000
70,6.476400
80,5.648900
90,4.772700
100,4.202100


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:226: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:226: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:226: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:226: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:22

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the fine-tuned T5 model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('/content/drive/MyDrive/t5')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

# Function to generate answer for given question
def generate_answer(question):
    input_text = 'question: ' + question.strip().lower()
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    outputs = model.generate(input_ids=input_ids, max_length=512, do_sample=True)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer
question = 'Give names of all products?'
answer = generate_answer(question)
print(answer)



/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


SELECT Name FROM table WHERE Product = all
